[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **LinkedIn Post Generator Agent**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

This notebook implements an automated LinkedIn post generator using CrewAI framework. The system uses two specialized agents:
1. Content Extractor Agent - Uses Gemini model to extract and summarize blog content
2. LinkedIn Post Writer Agent - Uses Gemini model to create engaging LinkedIn posts

## Installation
First, install the required packages:

In [ ]:
# !pip install crewai python-dotenv requests beautifulsoup4
# !pip install "crewai[tools]"


## Project Overview and Dependencies

This section defines the core functionality, goals, and prerequisites for the LinkedIn Post Generator agent. The system is built using CrewAI framework and requires specific Python packages and API keys.

In [55]:
"""
======================================================================================
LinkedIn Post Creator Agent - CrewAI Cookbook
======================================================================================

Goal:
- Given a blog URL (published by your company), create a LinkedIn post
- Uses CrewAI and its tools
- Two Agents:
  1. Content Extractor Agent: summarizes or extracts key points using "gemini-2.5-flash"
  2. LinkedIn Post Writer Agent: writes the LinkedIn post using "gemma-3-27b"

Prerequisites:
- Python 3.8+
- pip install crewai python-dotenv requests beautifulsoup4
- Working API keys for LLMs (Gemini and Gemma/Gemma-equivalent)

======================================================================================
"""
# ======================================================================================
# 1. IMPORTS AND ENVIRONMENT SETUP
# ======================================================================================
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
import requests
from bs4 import BeautifulSoup

# Load environment variables from .env file
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
assert GEMINI_API_KEY is not None, "Missing GEMINI_API_KEY in .env"

## Content Fetching Function

This section implements the core blog content extraction functionality:
- Uses `requests` to fetch blog content
- Implements `BeautifulSoup` for HTML parsing
- Extracts meaningful text while filtering out navigation, sidebars, and scripts
- Includes fallback mechanisms for different blog structures
- Limits content length to maintain LLM context window

In [56]:

# ======================================================================================
# 2. HELPER FUNCTION - FETCH AND CLEAN BLOG TEXT FROM URL
# ======================================================================================
def fetch_blog_content(url: str) -> str:
    """
    Downloads and cleans up the main text content from a blog URL.
    Returns only main readable text (no nav/sidebar/scripts).
    """
    try:
        resp = requests.get(url, timeout=15)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        # Heuristic: combine visible <p> tags
        text = " ".join([p.get_text(separator=" ", strip=True) for p in soup.find_all("p")])
        # Fallback to whole body if far too short
        if len(text) < 200:
            text = soup.body.get_text(separator=" ", strip=True)
        return text[:4000]  # limit to keep LLM context manageable
    except Exception as e:
        return f"ERROR fetching blog content: {e}"

# For direct preview/testing
if __name__ == "__main__":
    test_url = "https://www.labellerr.com/blog/product-update-august-2025/"
    print("Sample Extraction:\n", fetch_blog_content(test_url)[:500])


Sample Extraction:
 Data Annotation Platform Comprehensive solution for efficient data labeling. Video Annotation Platform Advanced tools for dynamic video labeling. Text Annotation Platform Powerful tools for accurate text labeling. Annotation Services Professional data labeling by experts. Image Annotation Platform Efficient and scalable image annotation solution. Dicom Annotation Tools Precision tools for medical image annotations. Interactive Demo Explore the platform with an interactive tour. Product Demos Wat


## CrewAI Tool Definition

This section defines a custom tool for the CrewAI agent:
- Wraps the blog content extraction function with `@tool` decorator
- Makes the function accessible to the Content Extractor agent
- Provides a clear interface for URL content extraction

In [57]:

# ======================================================================================
# 3. TOOL FOR CREWAI AGENT TO USE CONTENT EXTRACTION
# ======================================================================================
@tool("Blog Content Extractor")
def extract_blog_content(url: str) -> str:
    "Extracts and cleans up readable text from a blog URL for summarization/LLMs."
    return fetch_blog_content(url)


## LLM Configuration

This section configures the Language Models for both agents:
1. Content Extractor LLM:
   - Uses Gemini 2.0 Flash Lite
   - Low temperature (0.1) for consistent, factual summaries

2. Post Writer LLM:
   - Uses Gemini 2.5 Flash
   - Higher temperature (0.2) for creative writing

In [58]:

# ======================================================================================
# 4. CONFIGURE LLMs FOR CREWAI
# ======================================================================================
# Extraction LLM:
extracting_llm = LLM(
    model="gemini/gemini-2.0-flash-lite",
    api_key=GEMINI_API_KEY,
    temperature=0.1
)

# Writing LLM:
writing_llm = LLM(
    model="gemini/gemini-2.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0.2  # Higher for more creative copywriting
)


## Agent Definitions

This section creates two specialized CrewAI agents:

1. Content Extractor Agent:
   - Role: Summarize and extract key points from blog posts
   - Uses custom blog content extraction tool
   - Optimized for accurate content understanding

2. LinkedIn Post Writer Agent:
   - Role: Create engaging social media content
   - Focuses on B2B/B2C communication
   - Includes engagement optimization and CTAs

In [59]:

# ======================================================================================
# 5. DEFINE AGENTS
# ======================================================================================
extractor_agent = Agent(
    role="Content Extractor",
    goal="Summarize and extract the main points from the company blog post URL.",
    backstory="Specialist in reading web articles and distilling their most important value, tone, and insights.",
    tools=[extract_blog_content],
    llm=extracting_llm,
    verbose=True
)

writer_agent = Agent(
    role="LinkedIn Post Writer",
    goal="Draft a compelling, engaging LinkedIn post based on the extracted blog summary.",
    backstory="Expert in business communication and B2B/B2C social media content, knows how to boost engagement, add value statements and CTAs.",
    llm=writing_llm,
    verbose=True
)


## Task Definitions

This section defines the sequential tasks for the workflow:

1. Content Extraction Task:
   - Processes the blog URL
   - Creates a concise summary
   - Maintains key points and message integrity

2. LinkedIn Post Writing Task:
   - Creates engaging post content (max 180 words)
   - Includes emojis and hashtags
   - Adds clear call-to-action
   - References company and industry trends

In [60]:

# ======================================================================================
# 6. DEFINE TASKS
# ======================================================================================
extract_task = Task(
    description="Extract and summarize the core idea and key points from the blog URL: {blog_url}. Output a concise but complete summary suitable for use by a professional content writer.",
    expected_output="Summary of the blog post's key points, supporting details, and core message.",
    agent=extractor_agent
)
write_task = Task(
    description="Write a polished, creative LinkedIn post (max 180 words) based on the provided blog summary. Use a professional yet approachable tone. End with a clear call to action.use emoji for points. Reference the company and relate the post to industry trends if possible. add hashtags relevant to the topic. add given blog link also",
    expected_output="Ready-to-use LinkedIn post copy.",
    agent=writer_agent
)


## Crew Assembly

This section creates the CrewAI workflow:
- Combines both agents into a coordinated team
- Sets up sequential process flow
- Enables verbose output for monitoring
- Establishes task dependencies and order

In [61]:

# ======================================================================================
# 7. CREATE THE CREW
# ======================================================================================
linkedin_writer_crew = Crew(
    agents=[extractor_agent, writer_agent],
    tasks=[extract_task, write_task],
    process=Process.sequential,
    verbose=True
)


## Execution

This section runs the complete workflow:
- Sets the target blog URL
- Provides progress feedback
- Optionally shows extracted content preview
- Executes the full agent workflow
- Displays the final LinkedIn post

In [ ]:

# ======================================================================================
# 8. RUN THE WORKFLOW
# ======================================================================================
if __name__ == "__main__":
    
    BLOG_URL = "https://www.labellerr.com/blog/product-update-august-2025/"
    
    
    print(f"\n{'='*80}\nStarting LinkedIn Post Creator\n{'='*80}")
    print(f"Blog URL: {BLOG_URL}\n")

    # Optionally preview the extracted text
    # blog_text = fetch_blog_content(BLOG_URL)
    # print(f"\n{'='*80}\nEXTRACTED BLOG CONTENT (FIRST 1000 CHARACTERS)\n{'='*80}")
    # print(blog_text[:1000])

    # --- full agent workflow ---
    result = linkedin_writer_crew.kickoff(inputs={"blog_url": BLOG_URL})
    print(f"\n{'='*80}\nLINKEDIN POST COPY\n{'='*80}\n")
    print(result)




Starting LinkedIn Post Creator
Blog URL: https://www.labellerr.com/blog/product-update-august-2025/


LINKEDIN POST COPY
Tired of video annotations drifting off-sync or struggling with complex workflows? 😩

At Labellerr, we're thrilled to announce our latest release, bringing unparalleled precision and efficiency to video annotation! This update is a game-changer for anyone working with dynamic data, ensuring cleaner training data and faster AI development.

Here’s what’s new to streamline your projects:
✨ **Frame-Accurate Playback:** Say goodbye to sync issues! Annotations now stick exactly where they should.
🎬 **Intuitive Keyframe Management:** Create, manage, and review keyframes directly on your timeline for seamless labeling.
🚀 **Smarter SAM2 Tracking:** Track *all* selected annotations in a single job, dramatically speeding up your workflow and cutting compute costs.
🎯 **Stable Bounding Box Workflows:** Confidently create and apply SAM2 to bounding box labels without interruptio